In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import numpy as np

In [ ]:
# connect to drive
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
%ls
%cd gdrive/My Drive/Colab Notebooks/FaceMaskCheck

In [ ]:
trainingImage = ImageDataGenerator().flow_from_directory(directory="dataset", target_size=(224, 224))
print(trainingImage.class_indices)
testingImage = ImageDataGenerator().flow_from_directory(directory="datatest", target_size=(224, 224))
print(testingImage.class_indices)

Found 3246 images belonging to 2 classes.
{'with_mask': 0, 'without_mask': 1}
Found 600 images belonging to 2 classes.
{'with_mask': 0, 'without_mask': 1}


In [ ]:
del model
model = Sequential()
# 2 conv + polling
model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(224, 224, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPool2D(strides=(2, 2)))
# 2 conv + polling
model.add(Conv2D(kernel_size=(3, 3), filters=128, padding="same", activation="relu"))
model.add(Conv2D(kernel_size=(3, 3), filters=128, padding="same", activation="relu"))
model.add(MaxPool2D(strides=(2, 2), pool_size=(2, 2)))
# 3 conv + polling
model.add(Conv2D(kernel_size=(3, 3), filters=256, padding="same", activation="relu"))
model.add(Conv2D(kernel_size=(3, 3), filters=256, padding="same", activation="relu"))
model.add(Conv2D(kernel_size=(3, 3), filters=256, padding="same", activation="relu"))
model.add(MaxPool2D(strides=(2, 2), pool_size=(2, 2)))
# 3 conv + polling
model.add(Conv2D(kernel_size=(3, 3), filters=512, padding="same", activation="relu"))
model.add(Conv2D(kernel_size=(3, 3), filters=512, padding="same", activation="relu"))
model.add(Conv2D(kernel_size=(3, 3), filters=512, padding="same", activation="relu"))
model.add(MaxPool2D(strides=(2, 2), pool_size=(2, 2)))
# 3 conv + polling
model.add(Conv2D(kernel_size=(3, 3), filters=512, padding="same", activation="relu"))
model.add(Conv2D(kernel_size=(3, 3), filters=512, padding="same", activation="relu"))
model.add(Conv2D(kernel_size=(3, 3), filters=512, padding="same", activation="relu"))
model.add(MaxPool2D(strides=(2, 2), pool_size=(2, 2)))
# full connected
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(units=4096,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(units=2, activation="softmax"))

In [ ]:
model.compile(optimizer=Adam(0.0001), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
# check sumary before training
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 56, 56, 256)      

In [ ]:
history = model.fit_generator(generator=trainingImage, validation_data= testingImage, validation_steps=10,epochs=20)
model.save("firstModel.h5")

Epoch 1/20
 22/102 [=====>........................] - ETA: 32s - loss: 0.9695 - accuracy: 0.5248

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


102/102 [==============================] - 47s 463ms/step - loss: 0.4743 - accuracy: 0.7908 - val_loss: 0.1386 - val_accuracy: 0.9594
Epoch 2/20
102/102 [==============================] - 46s 448ms/step - loss: 0.1743 - accuracy: 0.9412 - val_loss: 0.2298 - val_accuracy: 0.9094
Epoch 3/20
102/102 [==============================] - 46s 452ms/step - loss: 0.1748 - accuracy: 0.9430 - val_loss: 0.1876 - val_accuracy: 0.9500
Epoch 4/20
102/102 [==============================] - 46s 448ms/step - loss: 0.1190 - accuracy: 0.9566 - val_loss: 0.2516 - val_accuracy: 0.9500
Epoch 5/20
102/102 [==============================] - 46s 450ms/step - loss: 0.1207 - accuracy: 0.9581 - val_loss: 0.1579 - val_accuracy: 0.9312
Epoch 6/20
102/102 [==============================] - 46s 455ms/step - loss: 0.0999 - accuracy: 0.9658 - val_loss: 0.1435 - val_accuracy: 0.9438
Epoch 7/20
102/102 [==============================] - 46s 449ms/step - loss: 0.1054 - accuracy: 0.9649 - val_loss: 0.2616 - val_accuracy: 0.9

## **Run this code to testing**

In [33]:
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
from google.colab import files
from google.colab import drive

drive.mount("/content/gdrive")
%cd gdrive/My Drive/Colab Notebooks/FaceMaskCheck

saved_model = load_model("mainModel.h5")

def predict(model, path):
  img = image.load_img(path,target_size=(224,224))
  img = np.asarray(img)
  img = np.expand_dims(img, axis=0)
  output = model.predict(img)
  if max(output[0][0], output[0][1]) < 0.5:
      return "Sorry i can't detect"
  if output[0][0] > output[0][1]:
      return "Have face mask"
  else:
      return 'Have no face mask'


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## This code use to check directly

In [ ]:
uploads = files.upload()
for file in uploads.keys():
  print("Predict " + file + " -> " + predict(saved_model, file))

# This code use to start a server to check

Install ngrok

In [ ]:
!pip install pyngrok
!ngrok authtoken 1hleG83gTnriU7FXYuCDvCfq6FW_isE8iBafyifAhA2jGFwG

Start Flask server

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import os
import base64


#start server ngrok
public_url = ngrok.connect(port = '8000')
print("Here is link: ", public_url)

#some CONST
UPLOAD_FOLDER = './uploadFiles'

#init and config
app = Flask(__name__)
app.secret_key = "12345"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# check if is image 

@app.route('/')
def hello_world():
    return render_template("takeWebCam.html")

@app.route('/upload', methods=['POST'])
def upload_file():
    # check if the post request has the file part
    data = request.form["file"].split(",")[1]
    imgdata = base64.b64decode(data)
    filename = os.path.join(UPLOAD_FOLDER, 'image.jpg')
    with open(filename, 'wb') as f:
        f.write(imgdata)
    return predict(saved_model, filename)

app.run(host='0.0.0.0', port=8000)

Here is link:  http://8269962dca87.ngrok.io
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Sep/2020 17:27:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 17:27:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Sep/2020 17:27:14] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 17:27:15] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 17:27:16] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 17:27:17] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 17:27:18] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 17:27:19] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 17:27:20] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 17:27:21] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 17:27:22] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 17:27:23] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 17:27:24] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 17:27:25] "POST /upload HTTP/1.1" 200 -
127.0.0